# Регулярные выражения и парсинг социальных сетей

## 1. Регулярные выражения

Теперь начнем с регулярных выражений. Выучить регулярки можно за 10 минут:

переходим **СЮДА**: https://regexone.com/.

Особым любителям регулярок [сюда](https://alf.nu/RegexGolf).

Обязательно к прочтению дома:

* [Регулярные выражения, пособие для новичков. Часть 1](https://habr.com/ru/post/115825/)

* [Регулярные выражения, пособие для новичков. Часть 2](https://habr.com/ru/post/115436/)

* https://ru.wikipedia.org/wiki/Регулярные_выражения

Загружаем библиотеку для регулярных выражений:

In [ ]:
import re

Правильный подход к вычислению регулярных выражений длинный и поэтапный:

 1. "Компиляция" регулярного выражения. На этом этапе "шаблон" преобразуется в сложный объект языка, содержащий в себе механизмы эффективного поиска ДАННОГО шаблона.
 2. "Применение" регулярного выражения к тексту. На этом этапе происходит обработка текста в объекте, созданном в пункте 1. 

1. Компиляция RE:

ничего удивительного, команда `re.compile()`, в скобках передаем само регулярное выражение

In [ ]:
pattern = re.compile(r"[А-ЯЁ][а-яё]+")  # что означает эта регулярка?

2. Применение регулярного выражения:

In [ ]:
text = "кот Василий выпил компот, а кот Иван выпил лимонад"

result = pattern.findall(text)
print(result)

Все тоже самое можно делать в одну строку:

In [ ]:
result2 = re.findall(r"[А-ЯЁ][а-яё]+", text)
print(result2)

Кроме `re.findall()` можно использовать `re.match()`, `re.finditer()`. Их можно изучить самостоятельно [тут](https://habr.com/ru/post/115825/). Мы же с вами рассмотрим автоматическую замену и деление по регулярному выражению.

1. Замена подстрок, найденных регуляркой на другую подстроку.

Команда: `re.sub("что заменяем", "на что заменяем", "где заменяем")`

Например, мы скачали много постов с ссылками и хотим эти ссылки удалить. А в тексте оставить строку "[здесь была ссылка]", чтобы мы всегда могли подсчитать, сколько ссылок было изначально.

In [ ]:
text = """13 сентября будем ходить ходуном. Прямо во дворе Пауэрхауса. Всё, как обычно, только соскучившись. Новые песни, старые песни. Прыжки и кувырки. Радость и смех.
Такое надо в корне пресекать!
Билеты: https://sbp4band.ticketscloud.org

Пожалуйста, планируйте приобретение билетов заранее. Высока вероятность, что продажа на входе осуществляться не будет"""

print(text)

In [ ]:
text = re.sub(r"https://.+\b", "[здесь была ссылка]", text)

print(text)

А теперь, наконец-таки, мы научимся ПРАВИЛЬНО делить слова на предложения:

In [ ]:
[i.strip() for i in re.split(r"(.+?[\.!\?]{1,}\s)", text) if i.strip()]

_NB! Безусловно, это неидеальная регулярка для деления текста на предложения. Подумайте, как вы можете ее улучшить и какие ситуации она не покрывает?_

Существует регулярное выражение, позволяющее делить текст на слова (ну, точнее, находить слова в тексте):

In [ ]:
re.findall(r"\w+", text)

**Задание:** напишите регулярное выражение, извлекающее все способы написания фамилии философа:

In [ ]:
philosophy = """Киркегор -датский философ, богослов и писатель, один из предшественников экзистенциализма.
С. Кьеркегор окончил теологический факультет Копенгагенского университета в 1840 году. 
Степень магистра получил в 1841 году, защитив диссертацию “О понятии иронии, с постоянным обращением к Сократу”, посвященную концепциям иронии у древнегреческих авторов и романтиков. 
Работы С. Кьеркегора отличаются исключительной психологической точностью и глубиной. 
Вклад в развистие философии, сделанный Кьеркегаардом. неоценим.
Сёрен Киркегаард: немецкое издание Сёрена Киркегаарда.
Спецкурс “С. Керкегор и история христианства в XIX в.” посвящен датскому философу Серену Керкегору."""

In [ ]:
# your code here

Самая короткая регулярка прибавляет 1 балл ко второй домашке (1 балл из финальных 10, 11 баллов за домашку тоже возможно)

#  2. Социальные сети

Вернемся к нашим баранам с прошлого семинара:

## VK

In [ ]:
import json
import vk

In [ ]:
token = ""  # Сервисный ключ доступа
session = vk.Session(access_token=token)  # Авторизация
vk_api = vk.API(session)

Под рукой всегда должна быть документация https://vk.com/dev/methods

#### Найти участников сообщества:

In [ ]:
def get_members(groupid, fields=('sex', 'bdate', 'city', 'country', 'domain')):
    first = vk_api.groups.getMembers(group_id=groupid, fields=fields, v=5.92)  # Первое выполнение метода
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"] // 1000  # Присваиваем переменной количество тысяч участников
    # С каждым проходом цикла смещение offset увеличивается на тысячу
    # и еще тысяча id'шников добавляется к нашему списку.
    for i in range(1, count+1):  
        data = data + vk_api.groups.getMembers(group_id=groupid, fields=fields, v=5.92, offset=i*1000)["items"]
    return data

#### Выгрузить посты из сообщества:

In [ ]:
def get_posts(owner_id):
    first = vk_api.wall.get(domain=owner_id, count=100, filter="owner", v=5.124)
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"] // 100  # Присваиваем переменной количество тысяч участников
    # С каждым проходом цикла смещение offset увеличивается на тысячу
    # и еще тысяча id'шников добавляется к нашему списку.
    for i in range(1, count+1):  
        data += vk_api.wall.get(domain=owner_id, v=5.124, filter="owner", count=100, offset=i*100)["items"]
    return [i['text'] for i in data]

#### Поиск пабликов по запросу:

In [ ]:
# https://vkhost.github.io/

personal_token = ""
session = vk.Session(access_token=personal_token)  # Авторизация
vk_personal_api = vk.API(session)  # Авторизация

Обратите внимание — даже при использовании параметра offset для получения информации доступны только первые 1000 результатов.

In [ ]:
def get_groups(query):
    first = vk_personal_api.groups.search(q=query, count=1000, type="group", v=5.124)
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"]
    return [(i['name'], i['id']) for i in data]

In [ ]:
found_groups = get_groups("hip-hop")
len(found_groups)

Великолепно. Мы собрали информацию по подписчикам, выгрузили все посты паблика и умеем искать паблики по запросу.

Теперь наша задача -- почистить данные с помощью регулярок и сохранить в файлы.

**Немного про ваш проект:** предполагается, что вы таким образом (или вручную) ищете и находите нужные вам паблики, далее для каждого паблика выгружаете нужную информацию: например, подписчиков (смотрите, из какого они города, их пол и возраст, строите графики) и посты (извлекаете информацию о концертах или об альбомах, клипах, и т.д.)

#### 1. get_groups

Для начала разберемся с найденным списком групп, сохраним его в файл:

In [ ]:
my_groups = get_groups("французский")

In [ ]:
with open("found_vk_groups.tsv", 'w', encoding="utf-8") as f:
    for group_name, group_id in my_groups:
        f.write(f"{group_name}\t{group_id}\n")

Теперь вы всегда можете его заново открыть и использовать в дальнейшем

#### 2. get_members

Теперь разберемся с подписчиками

In [ ]:
sbp4_members = get_members("sbp4band")
len(sbp4_members)

In [ ]:
with open("members.tsv", 'w', encoding='utf-8') as f:
    for member in sbp4_members:
        # our code here

#### 3. get_posts

и, наконец, с постами

In [ ]:
sbp4_posts = get_posts("sbp4band")
len(sbp4_posts)

In [ ]:
from pymystem3 import Mystem

In [ ]:
# your regexps below
http_pattern = re.compile(r"https://.+\b")
user_tag_pattern = re.compile(r"\[.+?\]")

In [ ]:
def clean_text(text):
    """
    удалить ссылки, удалить отметки пользователей, удалить переводы строк
    """
    text = re.sub(http_pattern, "WEBLINK", text)
    text = re.sub(user_tag_pattern, "USERTAG", text)
    text = text.replace("\n", " ")
    return text

In [ ]:
with open("texts.txt", 'w', encoding="utf-8") as f:
    for post in sbp4_posts:
        cleaned = clean_text(post)
        f.write(cleaned+"\n")

**Задание:** Лемматизируйте тексты и запишите их в файл `lemmatized_texts.txt` (каждый пост на отдельной строке):

In [ ]:
def lemmatize_text(text):
    m = Mystem()
    lemmas = m.lemmatize(text)
    lemmatized_text = ''.join(lemmas)
    return lemmatized_text

In [ ]:
# your code here